In [1]:
#This notebook will contain functions to build and update a list of mechs current in Mechwarrior Online.
#Mech list pages can be found at:
#Light: https://wiki.mwomercs.com/index.php?title=Light_Mechs&action=edit
#Medium: https://wiki.mwomercs.com/index.php?title=Medium_Mechs&action=edit
#Heavy:https://wiki.mwomercs.com/index.php?title=Heavy_Mechs&action=edit
#Assault: https://wiki.mwomercs.com/index.php?title=Assault_Mechs&action=edit

#all mechs via gamepedia: https://mwo.gamepedia.com/Category:Playable

In [167]:
import requests
from requests import get
import re
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd

In [4]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [133]:
#raw_html = simple_get('https://mwo.gamepedia.com/Category:Playable')
raw_html = simple_get("https://wiki.mwomercs.com/index.php?title=Light_Mechs&action=edit")
len(raw_html)
33878

33878

In [134]:
html = BeautifulSoup(raw_html, 'html.parser')


In [139]:
print(html)


<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>View source for Light Mechs - MWO Wiki</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Light_Mechs","wgTitle":"Light Mechs","wgCurRevisionId":1352,"wgRevisionId":0,"wgArticleId":101,"wgIsArticle":false,"wgIsRedirect":false,"wgAction":"edit","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":true,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["",

In [132]:
for i, td_class in enumerate(html):
    print(i, td_class)

0 html
1 

2 <html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Playable - MechWarrior Online Wiki</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":14,"wgPageName":"Category:Playable","wgTitle":"Playable","wgCurRevisionId":9701,"wgRevisionId":9701,"wgArticleId":2151,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Mechs"],"wgBreakFrames":true,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"

In [ ]:
### attempt lxml parsing

In [108]:
url='https://mwo.gamepedia.com/Category:Playable'
#has 4 tables, 1 for each weight class
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
#tr_elements = [ele for ele in tr_elements if len(ele)==14] #remove entries where entries != 14

In [109]:
#[len(T) for T in tr_elements]
len(tr_elements)

291

In [123]:
#table 0 contains all data for the table
#table 1 is v . d. e, hardpoints table (header?)
#table 2 is blank
#table 3 is entire table of mechs
#table 4 is column headers
#table 5 is blank
#table 6-70 are assault mech rows
#table 71 is blank
#table 72- are heavy mech rows, these are not split the same as assualt rows
#table 73 is header rows
# 74 is blank
# 75 - 136 are heavy mechs with 1 row per mech
# 138 header row and medium data
text_data = []
for i, t in enumerate(tr_elements[3]):
    print(i, "*"*100,"\n", t.text_content())
    text_data.append(str(t.text_content()))

0 **************************************************************************************************** 
 Assault BattleMechs
1 **************************************************************************************************** 
 
Assault BattleMechs

BattleMech
Weight
RA
RT
HD
CT
LT
LA
Total
JJ
MechModule
WeapModule
ConsModule
ECM


















Atlas AS7-BH
100
3
1
 
 
1
3
611
No
1
3
2
No


Atlas AS7-D
100
1
2
 
2
2
1
422
No
2
2
2
No


Atlas AS7-D-DC
100
1
2
 
 
3
1
223
No
1
1
2
Yes


Atlas AS7-K
100
1
1
 
2
1
1
411
No
2
2
2
No


Atlas AS7-RS
100
2
1
 
 
2
2
412
No
2
2
2
No


Awesome AWS-8Q
80
2
2
1
 
2
 
700
No
1
3
2
No


Awesome AWS-8R
80
2
2
1
 
2
 
304
No
1
2
2
No


Awesome AWS-8T
80
2
1
1
 
1
2
502
No
1
2
2
No


Awesome AWS-8V
80
1
3
1
 
2
 
403
No
1
2
2
No


Awesome AWS-9M
80
1
1
1
2  2
1
1
603
No
1
2
2
No


Awesome AWS-PB
80
1
1
1
 
1
2
303
No
1
2
2
No


Banshee BNC-3E
95
 
3
1
 
4
 
440
No
1
3
2
No


Banshee BNC-3M
95
 
4
1
 
3
 
800
No
1
2
2
No


Banshee BNC-3S
95
 
5  1


In [149]:
cols = []
new_row = []
text_data_test = [text.strip().replace("\n"," ") for text in text_data[1:]] #remove newlines
text_data_test = text_data_test[0].split() #convert list of one string into componenet elements
for i, data in enumerate(text_data_test):
    new_row.append(data)
    if i % 14 == 0:
        cols.append(new_row)
        new_row = []
#text_data_test
cols

[['Assault'],
 ['BattleMechs',
  'BattleMech',
  'Weight',
  'RA',
  'RT',
  'HD',
  'CT',
  'LT',
  'LA',
  'Total',
  'JJ',
  'MechModule',
  'WeapModule',
  'ConsModule'],
 ['ECM',
  'Atlas',
  'AS7-BH',
  '100',
  '3',
  '1',
  '1',
  '3',
  '611',
  'No',
  '1',
  '3',
  '2',
  'No'],
 ['Atlas',
  'AS7-D',
  '100',
  '1',
  '2',
  '2',
  '2',
  '1',
  '422',
  'No',
  '2',
  '2',
  '2',
  'No'],
 ['Atlas',
  'AS7-D-DC',
  '100',
  '1',
  '2',
  '3',
  '1',
  '223',
  'No',
  '1',
  '1',
  '2',
  'Yes',
  'Atlas'],
 ['AS7-K',
  '100',
  '1',
  '1',
  '2',
  '1',
  '1',
  '411',
  'No',
  '2',
  '2',
  '2',
  'No',
  'Atlas'],
 ['AS7-RS',
  '100',
  '2',
  '1',
  '2',
  '2',
  '412',
  'No',
  '2',
  '2',
  '2',
  'No',
  'Awesome',
  'AWS-8Q'],
 ['80',
  '2',
  '2',
  '1',
  '2',
  '700',
  'No',
  '1',
  '3',
  '2',
  'No',
  'Awesome',
  'AWS-8R',
  '80'],
 ['2',
  '2',
  '1',
  '2',
  '304',
  'No',
  '1',
  '2',
  '2',
  'No',
  'Awesome',
  'AWS-8T',
  '80',
  '2'],
 ['1',
  '

In [ ]:
tr_elements = doc.xpath('//tr')
use_cols = ["BattleMech", "Weight", "RA", "RT", "HD", "CT", "LT", "LA", "Total", "JJ", "MechModule", 
            "WeapModule", "ConsModule","ECM"]
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements: #0 is the header row
    i+=1
    name=t.text_content()
    print(i, "*"*50, "\n", name)
    if name in use_cols:
        print('{idx}:"{name}"'.format(idx=i, name=name))
        col.append((name,[]))

In [150]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=10:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [162]:
# use official pages

In [159]:
url='https://wiki.mwomercs.com/index.php?title=Light_Mechs&action=edit'
#has 4 tables, 1 for each weight class
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
#tr_elements = [ele for ele in tr_elements if len(ele)==14] #remove entries where entries != 14
tr_elements

[]

In [157]:
#table 0 contains all data for the table
#table 1 is v . d. e, hardpoints table (header?)
#table 2 is blank
#table 3 is entire table of mechs
#table 4 is column headers
#table 5 is blank
#table 6-70 are assault mech rows
#table 71 is blank
#table 72- are heavy mech rows, these are not split the same as assualt rows
#table 73 is header rows
# 74 is blank
# 75 - 136 are heavy mechs with 1 row per mech
# 138 header row and medium data
text_data = []
for i, t in enumerate(tr_elements):
    print(i, "*"*100,"\n", t.text_content())
    text_data.append(str(t.text_content()))

In [158]:
cols = []
new_row = []
text_data_test = [text.strip().replace("\n"," ") for text in text_data[1:]] #remove newlines
text_data_test = text_data_test[0].split() #convert list of one string into componenet elements
for i, data in enumerate(text_data_test):
    new_row.append(data)
    if i % 14 == 0:
        cols.append(new_row)
        new_row = []
#text_data_test
cols

IndexError: list index out of range

In [220]:
page_string = page.text
#page_string = page_string.replace("\n", " ")
page_string

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>View source for Light Mechs - MWO Wiki</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Light_Mechs","wgTitle":"Light Mechs","wgCurRevisionId":1352,"wgRevisionId":0,"wgArticleId":101,"wgIsArticle":false,"wgIsRedirect":false,"wgAction":"edit","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":true,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesSho

In [316]:

mech_obj = re.compile(r'===\s[\w\s]+\s[()A-Z0-9-]*\s===')
tonnage_obj = re.compile(r'Tonnage[\']+:[\s\d+]+')
chasis_obj = re.compile(r'Variants[\']+:[\sa-zA-Z0-9-,]+')
#get matching tonnage, variant list
mech_results = mech_obj.finditer(page_string)
tonnage_results = tonnage_obj.finditer(page_string)
chasis_results = chasis_obj.finditer(page_string)
                        
light_mech_weights = [mech_weight.group().replace("\n", "")[-3:].strip() for mech_weight in tonnage_results]
light_mechs = [mech_name.group().replace("===", "").strip() for mech_name in mech_results]
chasis_variants = [chasis.group().replace("\n","")[12:].replace(",","").split() for chasis in chasis_results]
for i in chasis_variants:
    print(i)

['FLE-15', 'FLE-17', 'FLE-19', 'FLE-20', 'FLE-FA']
['LCT-1E', 'LCT-1M', 'LCT-1V', 'LCT-3M', 'LCT-3S', 'LCT-3V']
['COM-1B', 'COM-1D', 'COM-2D', 'COM-3A']
['JVN-10F', 'JVN-10N', 'JNV-10P', 'JVN-11A', 'JVN-11B']
['OSR-1P', 'OSR-1V', 'OSR-2V', 'OSR-3D', 'OSR-4D']
['SDR-5D', 'SDR-5K', 'SDR-5V']
['UM-R60', 'RM-R60L', 'UM-R63', 'UM-R68']
['FS9-A', 'FS9-H', 'FS9-K', 'FS9-S']
['JR7-D', 'JR7-F', 'JR7-K']
['PNT-8Z', 'PNT-9R', 'PNT-10K', 'PNT-10P']
['RVN-2X', 'RVN-3L', 'RVN-4X']
['WLF-1', 'WLF-1A', 'WLF-1B', 'WLF-2']
['PIR-1', 'PIR-2', 'PRI-3', 'PIR-A', 'PIR-B']
['MLX-Prime', 'MLX-A', 'MLX-B', 'MLX-C', 'MLX-D', 'MLX-G']
['ACH-Prime', 'ACH-A', 'ACH-B', 'ACH-C', 'ACH-E']
['KFX-Prime', 'KFX-C', 'KFX-D', 'KFX-G', 'KFX-S']
['INC-1', 'INC-2', 'INC-3', 'INC-4', 'INC-5']
['ADR-Prime', 'ADR-A', 'ADR-B', 'ADR-D']
['COU-Prime', 'COU-C', 'COU-D', 'COU-E', 'COU-H']
['JR7-IIC', 'JR7-IIC-2', 'JR7-IIC-3', 'JR7-IIC-A']


In [317]:
light_mech_dict = {
    "mechs":light_mechs,
    "tonnage":light_mech_weights,
    "variants":chasis_variants
}
light_mech_df = pd.DataFrame(light_mech_dict)
light_mech_df

,mechs,tonnage,variants
0,Flea (FLE),20,"[FLE-15, FLE-17, FLE-19, FLE-20, FLE-FA]"
1,Locust (LCT),20,"[LCT-1E, LCT-1M, LCT-1V, LCT-3M, LCT-3S, LCT-3V]"
2,Commando (COM),25,"[COM-1B, COM-1D, COM-2D, COM-3A]"
3,Javelin (JVN),30,"[JVN-10F, JVN-10N, JNV-10P, JVN-11A, JVN-11B]"
4,Osiris (OSR),30,"[OSR-1P, OSR-1V, OSR-2V, OSR-3D, OSR-4D]"
5,Spider (SDR),30,"[SDR-5D, SDR-5K, SDR-5V]"
6,Urbanmech (UM),30,"[UM-R60, RM-R60L, UM-R63, UM-R68]"
7,Firestarter (FS9),35,"[FS9-A, FS9-H, FS9-K, FS9-S]"
8,Jenner (JR7),35,"[JR7-D, JR7-F, JR7-K]"
9,Panther (PNT),35,"[PNT-8Z, PNT-9R, PNT-10K, PNT-10P]"


In [308]:
len(light_mechs)

16

In [304]:
chasis_variants
#firestarter
#jenner

[['FLE-15', 'FLE-17', 'FLE-19', 'FLE-20', 'FLE-FA'],
 ['LCT-1E', 'LCT-1M', 'LCT-1V', 'LCT-3M', 'LCT-3S', 'LCT-3V'],
 ['COM-1B', 'COM-1D', 'COM-2D', 'COM-3A'],
 ['JVN-10F', 'JVN-10N', 'JNV-10P', 'JVN-11A', 'JVN-11B'],
 ['OSR-1P', 'OSR-1V', 'OSR-2V', 'OSR-3D', 'OSR-4D'],
 ['SDR-5D', 'SDR-5K', 'SDR-5V'],
 ['UM-R60', 'RM-R60L', 'UM-R63', 'UM-R68'],
 ['FS9-A', 'FS9-H', 'FS9-K', 'FS9-S'],
 ['JR7-D', 'JR7-F', 'JR7-K'],
 ['PNT-8Z', 'PNT-9R', 'PNT-10K', 'PNT-10P'],
 ['RVN-2X', 'RVN-3L', 'RVN-4X'],
 ['WLF-1', 'WLF-1A', 'WLF-1B', 'WLF-2'],
 ['PIR-1', 'PIR-2', 'PRI-3', 'PIR-A', 'PIR-B'],
 ['MLX-P'],
 ['ACH-P'],
 ['KFX-P'],
 ['INC-1', 'INC-2', 'INC-3', 'INC-4', 'INC-5'],
 ['ADR-P'],
 ['COU-P'],
 ['JR7-IIC', 'JR7-IIC-2', 'JR7-IIC-3', 'JR7-IIC-A']]

In [302]:
len(light_mech_weights)

20